# Link + Libraries

from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Basic Libraries
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sb
from pathlib import Path
import matplotlib.pyplot as plt
import math
#import quandl
#from pandas_datareader import data as pdr
import yfinance as yf

# Data

1. For the French Fama 3-factor model, we download the data from Ken French's website https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html.

Weekly and Daily data.

In [2]:
def data_import(file_name):
    file_name += '.csv'
    data = pd.DataFrame(pd.read_csv('./'+file_name,index_col = 0))
        
    return data

def factor_df(df,colnames):
    df.columns = colnames
    df = df.iloc[:-2,:]
    df.index = pd.to_datetime(df.index, format='%Y%m%d')
    return df
    

In [3]:
FFD = factor_df(data_import('FF_Research_Data_Factors_daily'), ['Mkt-RF', 'SMB', 'HML', 'RF'])
FFW = factor_df(data_import('FF_Research_Data_Factors_weekly'), ['Mkt-RF', 'SMB', 'HML', 'RF'])

In [4]:
FFD.head()

,Mkt-RF,SMB,HML,RF
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009


In [5]:
FFW.head()

,Mkt-RF,SMB,HML,RF
1926-07-02,1.60,-0.62,-0.83,0.056
1926-07-10,0.36,-0.88,0.31,0.056
1926-07-17,1.01,0.59,-1.44,0.056
1926-07-24,-2.05,0.10,-0.18,0.056
1926-07-31,3.04,-1.82,-0.90,0.056


In [6]:
FFD.describe()

,Mkt-RF,SMB,HML,RF
count,25438.000000,25438.000000,25438.000000,25438.000000
mean,0.029862,0.004672,0.015250,0.012098
std,1.082693,0.591557,0.624223,0.011885
min,-17.440000,-11.630000,-6.020000,-0.003000
25%,-0.400000,-0.250000,-0.250000,0.001000
50%,0.060000,0.020000,0.010000,0.010000
75%,0.500000,0.270000,0.260000,0.020000
max,15.760000,8.180000,8.820000,0.061000


In [7]:
FFW.describe()

,Mkt-RF,SMB,HML,RF
count,5043.000000,5043.000000,5043.000000,5043.000000
mean,0.146877,0.032040,0.080668,0.066481
std,2.473022,1.269348,1.507365,0.062532
min,-18.850000,-10.050000,-12.020000,-0.016000
25%,-1.050000,-0.615000,-0.620000,0.008000
50%,0.310000,0.010000,0.020000,0.055000
75%,1.410000,0.670000,0.700000,0.104000
max,17.040000,9.780000,15.390000,0.335000


2. EFT data

We download the ETF data from yahoo finance. We will need the adj close price for our strategy and backtesting. 


In [8]:
start_date = "2007-03-01"
end_date = "2023-02-28"

ticker_list = ['FXE', 'EWJ', 'GLD', 'QQQ', 'SPY','SHV', 'DBA', 'USO', 'XBI', 'ILF', 'EPP', 'FEZ']
ETF_price = pd.DataFrame(columns=ticker_list)

spy_ticker = 'SPY'

spy_price = pd.DataFrame(yf.download(spy_ticker,start=start_date, end=end_date))['Adj Close']

for val in ticker_list:
    df_val = pd.DataFrame(yf.download(val,start=start_date, end=end_date))
    ETF_price[val] = df_val['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
spy_return = spy_price.pct_change(1).dropna()
spy_return

Date
2007-03-02   -0.013095
2007-03-05   -0.009519
2007-03-06    0.017109
2007-03-07   -0.001002
2007-03-08    0.008455
                ...   
2023-02-21   -0.020061
2023-02-22   -0.001378
2023-02-23    0.005319
2023-02-24   -0.010682
2023-02-27    0.003406
Name: Adj Close, Length: 4026, dtype: float64

In [13]:
ETF_return = ETF_price.pct_change(1).dropna()
ETF_return

,FXE,EWJ,GLD,QQQ,SPY,SHV,DBA,USO,XBI,ILF,EPP,FEZ
Date,,,,,,,,,,,,
2007-03-02,0.000682,-0.016337,-0.032057,-0.015071,-0.013095,0.000551,-0.009384,-0.004489,-0.024478,-0.020606,-0.010886,-0.013192
2007-03-05,-0.007576,-0.014533,-0.012243,-0.007768,-0.009520,-0.000184,-0.001895,-0.027250,-0.008869,-0.022896,-0.030768,-0.017952
2007-03-06,0.002672,0.023877,0.019387,0.016608,0.017110,0.000092,-0.005315,0.014107,0.012222,0.049272,0.041028,0.028393
2007-03-07,0.004415,-0.002058,0.002338,-0.003968,-0.001002,0.000366,0.009160,0.019277,-0.003881,-0.008389,-0.003424,-0.001892
2007-03-08,-0.003183,0.008935,0.002799,0.006795,0.008455,0.000092,-0.000756,0.000000,0.004112,0.020025,0.012063,0.010231
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-21,-0.004863,-0.012276,-0.003737,-0.023675,-0.020061,0.000000,0.009766,-0.004924,-0.046429,-0.013810,-0.009613,-0.015707
2023-02-22,-0.003767,-0.007457,-0.005626,0.000748,-0.001378,0.000272,-0.004836,-0.026541,0.010390,-0.001236,-0.002032,-0.004163
2023-02-23,0.000204,0.010018,-0.000530,0.008734,0.005319,0.000363,-0.000972,0.022489,-0.001913,0.009485,-0.005429,0.010450


In [16]:
ETF_return.describe()

,FXE,EWJ,GLD,QQQ,SPY,SHV,DBA,USO,XBI,ILF,EPP,FEZ
count,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000,4026.000000
mean,-0.000041,0.000136,0.000296,0.000613,0.000420,0.000037,0.000006,-0.000164,0.000622,0.000293,0.000305,0.000236
std,0.005869,0.013189,0.011119,0.014316,0.012918,0.000222,0.010753,0.023853,0.020004,0.021747,0.016455,0.017549
min,-0.030749,-0.104077,-0.087808,-0.119788,-0.109424,-0.003620,-0.086060,-0.253150,-0.123472,-0.194667,-0.133405,-0.124643
25%,-0.003374,-0.006086,-0.005142,-0.005235,-0.004226,-0.000090,-0.005313,-0.011788,-0.010506,-0.010277,-0.006750,-0.007218
50%,0.000000,0.000444,0.000497,0.001146,0.000657,0.000000,0.000000,0.000731,0.000930,0.001031,0.000661,0.000855
75%,0.003273,0.006714,0.005893,0.007537,0.006000,0.000091,0.005604,0.012286,0.011899,0.011118,0.007692,0.008356
max,0.036716,0.158220,0.112905,0.121648,0.145198,0.003179,0.066724,0.166667,0.132641,0.262458,0.165901,0.175345


In [17]:
spy_return.describe()

count    4026.000000
mean        0.000420
std         0.012918
min        -0.109424
25%        -0.004226
50%         0.000657
75%         0.006000
max         0.145198
Name: Adj Close, dtype: float64